In [1]:
# chargement des données
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from scipy.stats import chi2_contingency
import scipy.stats as stats
import os
import math

import warnings
warnings.filterwarnings("ignore")

In [2]:
hackhaton=pd.read_csv("hackhaton_logic_replaced_fin.csv",encoding='ISO-8859-1')
hackhaton.drop('Unnamed: 0', axis =1, inplace=True)
hackhaton.drop('Unnamed: 0.1', axis =1, inplace=True)
hackhaton

,ID_CARRIERE,date_debut,date_fin,ETAT,CATEGORIE_EMPLOI,QUALITE,STATUT_AGENT,INDICE_BRUT,EMPLOI_STATUTAIRE,LIBELLE_EMPLOI_GRADE
0,56,01/01/1999,26/03/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
1,56,27/03/1999,16/07/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
2,56,17/07/1999,31/12/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
3,56,01/01/2000,02/07/2000,1,A,T,T,NaN,NaN,adjointadministratif.........
4,56,03/07/2000,19/07/2000,1,A,T,T,NaN,NaN,adjointadministratif.........
...,...,...,...,...,...,...,...,...,...,...
1048570,1861885,01/02/2009,28/02/2009,1,A,T,T,303.0,NaN,adjpat2c
1048571,1861885,01/03/2009,31/03/2009,1,A,T,T,303.0,NaN,adjpat2c
1048572,1861885,01/04/2009,08/04/2009,1,A,T,T,303.0,NaN,adjpat2c
1048573,1861885,09/04/2009,19/04/2009,1,A,T,T,303.0,NaN,adjpat2c


# Normalisation des libellés (pas de caractères spéciaux, ni d'espaces, ni de voyelles)

In [3]:
hackhaton2=hackhaton

In [4]:
hackhaton2

,ID_CARRIERE,date_debut,date_fin,ETAT,CATEGORIE_EMPLOI,QUALITE,STATUT_AGENT,INDICE_BRUT,EMPLOI_STATUTAIRE,LIBELLE_EMPLOI_GRADE
0,56,01/01/1999,26/03/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
1,56,27/03/1999,16/07/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
2,56,17/07/1999,31/12/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
3,56,01/01/2000,02/07/2000,1,A,T,T,NaN,NaN,adjointadministratif.........
4,56,03/07/2000,19/07/2000,1,A,T,T,NaN,NaN,adjointadministratif.........
...,...,...,...,...,...,...,...,...,...,...
1048570,1861885,01/02/2009,28/02/2009,1,A,T,T,303.0,NaN,adjpat2c
1048571,1861885,01/03/2009,31/03/2009,1,A,T,T,303.0,NaN,adjpat2c
1048572,1861885,01/04/2009,08/04/2009,1,A,T,T,303.0,NaN,adjpat2c
1048573,1861885,09/04/2009,19/04/2009,1,A,T,T,303.0,NaN,adjpat2c


In [5]:
hackhaton2['LIBELLE_EMPLOI_GRADE']=[str(i) for i in hackhaton2['LIBELLE_EMPLOI_GRADE']]

In [6]:
#on enleve les voyelles des libellés ? (si jamais il y a des abreviations, souvent on saute les voyelles...) 
#On enleve aussi les espaces on sait jamais

voyelles='aeiouyÃÂ'

L=[]
for i in hackhaton2['LIBELLE_EMPLOI_GRADE'] :
    
    if i != 'nan' :
        m=''
        
        for j in range(0, len(i)):
            
            if i[j] in voyelles:  #Enlever les voyelles (pour les abréviations) 
                pass
            
            else :
                m = m + i[j]
        
        
    else :
        m=i
    
    m = "".join(char for char in m if char.isalnum()) #Enlever les caractères spéciaux
    L.append(m)

In [7]:
hackhaton2['LIBELLE_EMPLOI_GRADE']=[str(i) for i in L]

In [8]:
hackhaton2

,ID_CARRIERE,date_debut,date_fin,ETAT,CATEGORIE_EMPLOI,QUALITE,STATUT_AGENT,INDICE_BRUT,EMPLOI_STATUTAIRE,LIBELLE_EMPLOI_GRADE
0,56,01/01/1999,26/03/1999,1,A,T,T,NaN,NaN,djntdmnstrtf
1,56,27/03/1999,16/07/1999,1,A,T,T,NaN,NaN,djntdmnstrtf
2,56,17/07/1999,31/12/1999,1,A,T,T,NaN,NaN,djntdmnstrtf
3,56,01/01/2000,02/07/2000,1,A,T,T,NaN,NaN,djntdmnstrtf
4,56,03/07/2000,19/07/2000,1,A,T,T,NaN,NaN,djntdmnstrtf
...,...,...,...,...,...,...,...,...,...,...
1048570,1861885,01/02/2009,28/02/2009,1,A,T,T,303.0,NaN,djpt2c
1048571,1861885,01/03/2009,31/03/2009,1,A,T,T,303.0,NaN,djpt2c
1048572,1861885,01/04/2009,08/04/2009,1,A,T,T,303.0,NaN,djpt2c
1048573,1861885,09/04/2009,19/04/2009,1,A,T,T,303.0,NaN,djpt2c


# Fonction de ressemblance des libellés

## Séparation des libellés associés a des nan et les autres 

In [9]:
# liste des libellés associés a des 'nan'
ind_code_nan=[i for i in range (0,1048574) if str(hackhaton2.EMPLOI_STATUTAIRE[i]) == 'nan']
libelle_nan =[hackhaton2.LIBELLE_EMPLOI_GRADE[i] for i in ind_code_nan]
prout=libelle_nan
libelle_nan=list(set(libelle_nan))
len(libelle_nan)

7976

In [10]:
# liste des libellés  (unique) associés a des codes (dictionnaire ?)

ind_code_no_nan=[i for i in range (0,1048574) if str(hackhaton2.EMPLOI_STATUTAIRE[i]) != 'nan']
libelle_no_nan =[hackhaton2.LIBELLE_EMPLOI_GRADE[i] for i in ind_code_no_nan]
libelle_no_nan=list(set(libelle_no_nan))
len(libelle_no_nan)

7348

In [11]:
Liste=[]
for lib in libelle_no_nan : 
    Liste.append((lib,list(np.unique(list(hackhaton2[hackhaton2.LIBELLE_EMPLOI_GRADE == lib]['EMPLOI_STATUTAIRE'])))))
    

In [12]:
p=[(a,b) for a,b in dict(Liste).items() if ((len(b)==2) and ('nan' in b)) or (len(b)==1)]
p2=[(a,b[0]) if b[0]!='nan' else (a,b[1]) for a,b in p]

In [13]:
lib_to_onecode=[a for a,b in p2]

In [14]:
lib_to_onecode

['ttchddmnstrtnhsptlrprncpl',
 'gntspclsprncpl2m',
 'xlrdpr1cl',
 'pschmtrcndclssspct',
 'gntsrvhsptlrqlf2mct',
 'dsgnntdclx',
 'gntqlfptrmcls',
 'ttchdcnsrvtn',
 'lgr',
 'djntdjpnstrtf',
 'pschmtrcnclspc',
 'tchncndssrvcsprtnnlsn',
 'nfrmrdclnrmlbns',
 'cdrscdctftv',
 'ssdcns2cl',
 'dcppldjnsnfnts',
 'djntdmnstrtfprncpl2m',
 'ssstntscdctvprncpl',
 'djntdmprncpl1',
 'cncrgdjnttchnqppld2mclss',
 'drctrdjntdtblssmntpblcdsvllsd40m15m0hbtnts',
 'djnttchnqtrrtrlprncpld2mclss01022014',
 'gntspclsprncpl2clssclmtrn',
 'gnttchnqdjnttchnq',
 'gntdsrvcstchnqs',
 'gnttrrtrlptrmn',
 'gtslbrt',
 'dctrpspl2cl',
 'gntdbrgrp2',
 'xlrdprcltrtrrtrld1rclss',
 'djnttch2clnc',
 'cnsllrpplps2cl',
 'jrst',
 'gntdlgstqlng',
 'chfsrvcplcmncplprncpl2clss',
 'gntdsrvcslpplt',
 'dttcnn',
 'djnttchnqdjnttchnqd2',
 'djnttchnq2clst',
 'cnsllrdspstdlnmtn',
 'tsmppl2ndclss',
 'djnttchnqtrrtrlprncpldprmrclss',
 'ssstntmdcdmnstrtfclxc',
 'gntrtnqlf',
 'djntdmn',
 'rclsnfcdrdsnt',
 'ddrdlg',
 'tchncntrrtrlprncp',
 'rprclt

## Fonction de ressemblance

In [15]:
import difflib


#L=[corresondant de libelle_nan[i], si pas de correspondant, c'est libelle_nan[i]]
# de longueur len(libelle_nan) 
c1=0
c2=0
L_replacable=[]
L=[]
for i in range (0,len(libelle_nan)) :
    
    print(i+1,'/',len(libelle_nan))
    Match=difflib.get_close_matches(libelle_nan[i], lib_to_onecode, cutoff = 0.95) #On selectionne les libellé assigné a un code qui match a au moins 90% avec notre libellé sans code 

    if len(Match) != 0:
        c1=c1+1
        Match_ratio = [difflib.SequenceMatcher(None, libelle_nan[i], a).ratio() for a in Match]
        index_max= Match_ratio.index(max(Match_ratio))
        L_replacable.append(libelle_nan[i])
        L.append(Match[index_max]) #Match[index_max] est l'element le plus ressemblant a notre libelle de base
        
    else : 
        c2=c2+1
        L.append(libelle_nan[i])

1 / 7976
2 / 7976
3 / 7976
4 / 7976
5 / 7976
6 / 7976
7 / 7976
8 / 7976
9 / 7976
10 / 7976
11 / 7976
12 / 7976
13 / 7976
14 / 7976
15 / 7976
16 / 7976
17 / 7976
18 / 7976
19 / 7976
20 / 7976
21 / 7976
22 / 7976
23 / 7976
24 / 7976
25 / 7976
26 / 7976
27 / 7976
28 / 7976
29 / 7976
30 / 7976
31 / 7976
32 / 7976
33 / 7976
34 / 7976
35 / 7976
36 / 7976
37 / 7976
38 / 7976
39 / 7976
40 / 7976
41 / 7976
42 / 7976
43 / 7976
44 / 7976
45 / 7976
46 / 7976
47 / 7976
48 / 7976
49 / 7976
50 / 7976
51 / 7976
52 / 7976
53 / 7976
54 / 7976
55 / 7976
56 / 7976
57 / 7976
58 / 7976
59 / 7976
60 / 7976
61 / 7976
62 / 7976
63 / 7976
64 / 7976
65 / 7976
66 / 7976
67 / 7976
68 / 7976
69 / 7976
70 / 7976
71 / 7976
72 / 7976
73 / 7976
74 / 7976
75 / 7976
76 / 7976
77 / 7976
78 / 7976
79 / 7976
80 / 7976
81 / 7976
82 / 7976
83 / 7976
84 / 7976
85 / 7976
86 / 7976
87 / 7976
88 / 7976
89 / 7976
90 / 7976
91 / 7976
92 / 7976
93 / 7976
94 / 7976
95 / 7976
96 / 7976
97 / 7976
98 / 7976
99 / 7976
100 / 7976
101 / 79

In [16]:
print(c1 , 'de remplacés et ', c2, 'non remplacés sur en tout', c1+c2, 'libellés')

s = 0
for i in L_replacable : 
    s += prout.count(i)
print(s)

2817 de remplacés et  5159 non remplacés sur en tout 7976 libellés
152839


In [17]:
print((s/len(prout))*100, '% remplaçable')

38.025421767979715 % remplaçable


In [18]:
len(libelle_nan) == len(L) 
#Les deux listes pour creer le dictionnaire de correspondance entre libellé avec des nan et leur libellé (qui ressemble a 90% et avec un code) correspondant

True

In [19]:
correspondance=[{libelle_nan[i] : L[i]} for i in range (0,len(L))]

In [20]:
c=correspondance[0]
for i in range (1,len(correspondance)): 
    c.update(correspondance[i])


In [21]:
c

{'': '',
 'gntspcclsmt1clss': 'gntspcclsmt1clss',
 'xlrdpr1cl': 'xlrdpr1cl',
 'gntprvntn': 'gntprvntn',
 'gntdmnstrtf1rcl': 'gntdmnstrtf1rcl',
 'djntdmnsppl2clss': 'djntdmnsppl2clss',
 'scrtmdclclnrstg0': 'scrtmdclclnrstg0',
 'dsgnntdclx': 'dsgnntdclx',
 'gnttchnqdntrtnpr': 'gnttchnqdntrtnpr',
 'djtdmtrrtd1clss': 'djtdmtrrtd1clss',
 'xlrdprcltrcr': 'xlrdprcltrcr',
 'rdctrtrrtr': 'rdctrtrrtr',
 'ttchdcnsrvtn': 'ttchdcnsrvtn',
 'lgr': 'lgr',
 'gntch2ct': 'gntch2ct',
 'hmmdqpbr': 'hmmdqpbr',
 'tllncmtrnllsr': 'tllncmtrnllsr',
 'ngnrchfclxcpt': 'ngnrnchfclxcpt',
 'drtbsn': 'drtbsn',
 'cdrscdctftv': 'cdrscdctftv',
 'nfpschtrqcn': 'nfpschtrqcn',
 'djntdmnprncdxcl': 'djntdmprncdxcl',
 'mntrcjrdndnfnts': 'mntrcjrdndnfnts',
 'djntdmnstrtfprncpl2m': 'djntdmnstrtfprncpl2m',
 'ssstntscdctvprncpl': 'ssstntscdctvprncpl',
 'lvnfrmr': 'lvnfrmr',
 'rgthrptclsp': 'rgthrptclsp',
 'djntdmprncpl1': 'djntdmprncpl1',
 'vrrprfssnnllqlf': 'vrrprfssnnllqlf',
 'nfsrvchfsrvmd': 'nfsrvchfsrvmd',
 'djtdmnstrtfddxcl

## Application a la BDD

In [22]:
def replace(i, data):
    try:
        return data[i]
    except KeyError :
        return i

In [23]:
new_grade=[replace(i,c) for i in list(hackhaton2.LIBELLE_EMPLOI_GRADE)]

In [24]:
hackhaton2.LIBELLE_EMPLOI_GRADE=new_grade

In [25]:
hackhaton2

,ID_CARRIERE,date_debut,date_fin,ETAT,CATEGORIE_EMPLOI,QUALITE,STATUT_AGENT,INDICE_BRUT,EMPLOI_STATUTAIRE,LIBELLE_EMPLOI_GRADE
0,56,01/01/1999,26/03/1999,1,A,T,T,NaN,NaN,djntdmnstrtft
1,56,27/03/1999,16/07/1999,1,A,T,T,NaN,NaN,djntdmnstrtft
2,56,17/07/1999,31/12/1999,1,A,T,T,NaN,NaN,djntdmnstrtft
3,56,01/01/2000,02/07/2000,1,A,T,T,NaN,NaN,djntdmnstrtft
4,56,03/07/2000,19/07/2000,1,A,T,T,NaN,NaN,djntdmnstrtft
...,...,...,...,...,...,...,...,...,...,...
1048570,1861885,01/02/2009,28/02/2009,1,A,T,T,303.0,NaN,djpt2c
1048571,1861885,01/03/2009,31/03/2009,1,A,T,T,303.0,NaN,djpt2c
1048572,1861885,01/04/2009,08/04/2009,1,A,T,T,303.0,NaN,djpt2c
1048573,1861885,09/04/2009,19/04/2009,1,A,T,T,303.0,NaN,djpt2c


# Remplacement des codes si égalité des libellés 

In [26]:
ind_code_nan=[i for i in range (0,1048574+1) if str(hackhaton2.EMPLOI_STATUTAIRE[i]) == 'nan']
libelle_nan =[hackhaton2.LIBELLE_EMPLOI_GRADE[i] for i in ind_code_nan] #le premier libellés correspond à celui de la ligne ind_code_nan[i]
libelle_nan=list(set(libelle_nan))
len(libelle_nan)

7300

In [27]:
ind_code_no_nan=[i for i in range (0,1048574+1) if str(hackhaton2.EMPLOI_STATUTAIRE[i]) != 'nan']
libelle_no_nan =[hackhaton2.LIBELLE_EMPLOI_GRADE[i] for i in ind_code_no_nan] #le premier libellés correspond à celui de la ligne ind_code_no_nan[i]
libelle_no_nan_unic=list(set(libelle_no_nan))
print('len(libelle_no_nan) =',len(libelle_no_nan))
print('len(libelle_no_nan_unic) =',len(libelle_no_nan_unic))


len(libelle_no_nan) = 646635
len(libelle_no_nan_unic) = 6952


In [28]:
a=list(hackhaton2.LIBELLE_EMPLOI_GRADE)

for i in range (0,len(libelle_nan)) :
    
        print(i+1,'/',len(libelle_nan))
        #print(libelle_nan[i])
        Match=difflib.get_close_matches(libelle_nan[i], lib_to_onecode, cutoff = 1) #On selectionne les libellé assigné a un code qui est égal à notre libellé sans code 
        #print(Match)
        #print(libelle_nan[i] ,'match avec', Match[0])
        #print(len(Match))
        
        if len(Match) != 0 and Match[0]!='':
            
            #print(libelle_nan[i] ,'match avec', Match[0]) 
            position_nan =list(np.where(np.array(a) == libelle_nan[i])[0]) #position du libellé ou le code est nan dans hackhaton 
            #print('position des nan dans hackaton', position_nan)
            
            index_no_nan=ind_code_no_nan[libelle_no_nan.index(Match[0])] #cet index correspond au libellés de la ligne ind_code_no_nan[i]
            for j in position_nan : 
                if hackhaton2.EMPLOI_STATUTAIRE[j] != hackhaton2.EMPLOI_STATUTAIRE[index_no_nan]: 
                    
                    #print('on remplace', hackhaton.EMPLOI_STATUTAIRE[j], 'par', hackhaton2.EMPLOI_STATUTAIRE[index_no_nan])
                    hackhaton2.EMPLOI_STATUTAIRE[j]= hackhaton2.EMPLOI_STATUTAIRE[index_no_nan]

        else : 
            pass

1 / 7300
2 / 7300
3 / 7300
4 / 7300
5 / 7300
6 / 7300
7 / 7300
8 / 7300
9 / 7300
10 / 7300
11 / 7300
12 / 7300
13 / 7300
14 / 7300
15 / 7300
16 / 7300
17 / 7300
18 / 7300
19 / 7300
20 / 7300
21 / 7300
22 / 7300
23 / 7300
24 / 7300
25 / 7300
26 / 7300
27 / 7300
28 / 7300
29 / 7300
30 / 7300
31 / 7300
32 / 7300
33 / 7300
34 / 7300
35 / 7300
36 / 7300
37 / 7300
38 / 7300
39 / 7300
40 / 7300
41 / 7300
42 / 7300
43 / 7300
44 / 7300
45 / 7300
46 / 7300
47 / 7300
48 / 7300
49 / 7300
50 / 7300
51 / 7300
52 / 7300
53 / 7300
54 / 7300
55 / 7300
56 / 7300
57 / 7300
58 / 7300
59 / 7300
60 / 7300
61 / 7300
62 / 7300
63 / 7300
64 / 7300
65 / 7300
66 / 7300
67 / 7300
68 / 7300
69 / 7300
70 / 7300
71 / 7300
72 / 7300
73 / 7300
74 / 7300
75 / 7300
76 / 7300
77 / 7300
78 / 7300
79 / 7300
80 / 7300
81 / 7300
82 / 7300
83 / 7300
84 / 7300
85 / 7300
86 / 7300
87 / 7300
88 / 7300
89 / 7300
90 / 7300
91 / 7300
92 / 7300
93 / 7300
94 / 7300
95 / 7300
96 / 7300
97 / 7300
98 / 7300
99 / 7300
100 / 7300
101 / 73

In [29]:
hackhaton2.to_csv('hackhaton_replaced_final.csv')